In [1]:
import rdflib
from rdflib import URIRef, Graph, Namespace
from rdflib.namespace import OWL, RDF, RDFS

import ontospy
from ontospy.ontodocs.viz.viz_d3tree import *

import urllib.request

In [2]:
!pip show rdflib

Name: rdflib
Version: 5.0.0
Summary: RDFLib is a Python library for working with RDF, a simple yet powerful language for representing information.
Home-page: https://github.com/RDFLib/rdflib
Author: Daniel 'eikeon' Krech
Author-email: eikeon@eikeon.com
License: BSD-3-Clause
Location: /Users/Rohan/opt/anaconda3/lib/python3.7/site-packages
Requires: six, isodate, pyparsing
Required-by: SPARQLWrapper, rdflib-jsonld, ontospy


In [3]:
!pip show ontospy

Name: ontospy
Version: 1.9.8.3
Summary: Query, inspect and visualize knowledge models encoded as RDF/OWL ontologies.
Home-page: https://github.com/lambdamusic/ontospy
Author: Michele Pasin
Author-email: michele.pasin@gmail.com
License: MIT
Location: /Users/Rohan/opt/anaconda3/lib/python3.7/site-packages
Requires: rdflib, html5lib, rdflib-jsonld, SPARQLWrapper, click, keepalive, pyfiglet, requests, colorama, pyparsing
Required-by: 


In [4]:
target_url = 'https://raw.githubusercontent.com/aigents/aigents-java-nlp/master/src/test/java/org/aigents/nlp/lg\
/poc-english_5C_2018-06-06_0004.4.0.dict.txt'

In [5]:
text = []
for line in urllib.request.urlopen(target_url):
    line = line.decode('utf-8')
    text.append(line)

In [6]:
sets = []
for i, block in enumerate(text):
    if '% C' in block:
        cluster = text[i].replace('% ','').replace('\n','')
        a = text[i+2]
        cluster_words = text[i+1].replace(':\n','').replace('"','').split(' ')
        cluster_or_set = text[i+2].replace(';\n','').replace('(','').replace(')','').replace(' & ','&').split(' or ')
        sets.append([cluster, cluster_words, cluster_or_set])

In [7]:
myOntology =  Namespace("http://www.semanticweb.org/myOntology#")
g = Graph()
g.bind("myOntology", myOntology)
g = Graph()

In [8]:
for case in sets:
    cluster = URIRef(myOntology[case[0]])
    g.add((cluster, RDF.type, OWL.Class))
    g.add((cluster, RDFS.subClassOf, OWL.Thing))
    for w in case[1]:
        word = URIRef(myOntology[w])
        g.add((word, RDF.type, OWL.Class))
        g.add((word, RDFS.subClassOf, cluster))
    for orc in case[2]:
        or_case = URIRef(myOntology[orc])
        g.add((or_case, RDF.type, OWL.Class))
        g.add((or_case, RDFS.subClassOf, cluster))
        for ampc in orc.split('&'):
            amp_case = URIRef(myOntology[ampc])
            g.add((amp_case, RDF.type, OWL.Class))
            g.add((amp_case, RDFS.subClassOf, or_case))

In [9]:
g.bind("owl", OWL)
g.serialize(destination='output.owl', format='pretty-xml')

In [10]:
v = ontospy.Ontospy("output.owl")
v = Dataviz(v)
v.build()
v.preview()

Done.
=> file:///Users/Rohan/ontospy-viz-test/outputowl/index.html
